![](imgs/logo.png)

# Przetwarzanie Big Data z użyciem Apache Spark

Autor notebooka: Jakub Nowacki.


## Spark SQL - statystyki z użyciem metod DataFrame 

W katalogu `data/` znajduje się plik `rollingsales_bronx.csv`.

Poniżej analizujemy dane statystyczne domów: liczność, średnie powierzchnie, lata i ceny, grupując po dzielnicy i typie.

In [1]:
import pyspark
import pyspark.sql.functions as func
import pyspark.sql.types as types
sc = pyspark.SparkContext(appName='HouseStatsDF')
sqlContext = pyspark.sql.SQLContext(sc)

In [2]:
import re

# Użycie Row to jedna z metod tworzenia DataFrame z danych; nie wymaga dodatkowego schematu
# zobacz dokumentację: http://spark.apache.org/docs/latest/api/python/pyspark.sql.html
def from_csv(line):
    # Stałe podane są tylko dla czytelności
    HOOD_COLUMN = 1
    TYPE_COLUMN = 2;
    LAND_AREA_COLUMN = 14;
    GROSS_AREA_COLUMN = 15;
    YEAR_COLUMN = 16;
    PRICE_COLUMN = 19;
    # Dzielimy linię na kolumny i przypisujemy do kluczy w słowniku
    c = line.split(',')
    row = dict()
    row['hood'] = c[HOOD_COLUMN];
    row['type'] = c[TYPE_COLUMN];
    row['landArea'] = int(re.sub(r'[^\d]', '', c[LAND_AREA_COLUMN]));
    row['grossArea'] = int(re.sub(r'[^\d]', '', c[GROSS_AREA_COLUMN]));
    row['year'] = int(re.sub(r'[^\d]', '', c[YEAR_COLUMN]));
    row['price'] = int(re.sub(r'[^\d]', '', c[PRICE_COLUMN]));
    # Zwracamy obiekt Row
    return pyspark.Row(**row)

In [3]:
# Używamy powyższą metodę generującą obiekty Row do przeczytania CSV linia po linii
# Zauważ, że tworzymy zwykłe RDD
sales_rdd = sc.textFile('data/rollingsales_bronx.csv').map(lambda line: from_csv(line))
# Z RDD złożonego z obiektów Row możemy bezpośrednio stworzyć DataFrame bez dodatkowego schematu
sales = sales_rdd.toDF()
sales.printSchema()
sales.show()

root
 |-- grossArea: long (nullable = true)
 |-- hood: string (nullable = true)
 |-- landArea: long (nullable = true)
 |-- price: long (nullable = true)
 |-- type: string (nullable = true)
 |-- year: long (nullable = true)

+---------+--------------------+--------+------+--------------------+----+
|grossArea|                hood|landArea| price|                type|year|
+---------+--------------------+--------+------+--------------------+----+
|     2048|BATHGATE         ...|    1842|355000|01  ONE FAMILY HO...|1901|
|     1290|BATHGATE         ...|    1103|474819|01  ONE FAMILY HO...|1910|
|     1344|BATHGATE         ...|    1986|210000|01  ONE FAMILY HO...|1899|
|     1431|BATHGATE         ...|    2329|343116|01  ONE FAMILY HO...|1901|
|     4452|BATHGATE         ...|    1855|     0|02  TWO FAMILY HO...|1931|
|     2400|BATHGATE         ...|    2000|316500|02  TWO FAMILY HO...|1993|
|     2394|BATHGATE         ...|    2498|390000|02  TWO FAMILY HO...|1995|
|     1542|BATHGATE       

In [4]:
a1 = sales.groupBy('hood', 'type') \
    .agg(func.count('type').alias('count'),
         func.avg('landArea').alias('avgLandArea'),
         func.avg('year').alias('avgYear'),
         func.avg('price').alias('avgPrice')) \
    .orderBy('hood', 'type')
a1.show()

+--------------------+--------------------+-----+------------------+------------------+------------------+
|                hood|                type|count|       avgLandArea|           avgYear|          avgPrice|
+--------------------+--------------------+-----+------------------+------------------+------------------+
|BATHGATE         ...|01  ONE FAMILY HO...|    4|            1815.0|           1902.75|         345733.75|
|BATHGATE         ...|02  TWO FAMILY HO...|   10|            2131.1|            1947.2|          203427.6|
|BATHGATE         ...|03  THREE FAMILY ...|    6|            2252.5|            1919.0| 292019.1666666667|
|BATHGATE         ...|05  TAX CLASS 1 V...|    1|            2099.0|               0.0|           40730.0|
|BATHGATE         ...|07  RENTALS - WAL...|    6|3161.6666666666665|            1924.5| 424286.1666666667|
|BATHGATE         ...|10  COOPS - ELEVA...|    1|               0.0|               0.0|               4.0|
|BATHGATE         ...|14  RENTALS - 4

## Zadania

* Popraw wyniki usuwając błędne dane.
* Zastosuj odpowiednią prezentację dany w zależności od typu.
* Policz średnie dla domów z XX w. tylko dla grup zawierających więcej niż 10 domów.
* ★ Policz średnie tylko dla 10 najbogatszych dzielnic.

In [25]:
a1 = sales.where('landArea > 0 AND price > 0 AND year > 1500') \
    .where(sales.hood.like('%BAY%')) \
    .groupBy('hood', 'type') \
    .agg(func.count('type').alias('count'),
         func.avg('landArea').cast(types.DecimalType(16,2)).alias('avgLandArea'),
         func.avg('year').cast('integer').alias('avgYear'),
         func.avg('price').cast(types.DecimalType(16,2)).alias('avgPrice')) \
    .where('count >10') \
    .orderBy('hood', func.col('type').desc())
a1.show()

+--------------------+--------------------+-----+-----------+-------+---------+
|                hood|                type|count|avgLandArea|avgYear| avgPrice|
+--------------------+--------------------+-----+-----------+-------+---------+
|BAYCHESTER       ...|03  THREE FAMILY ...|   17|    2522.65|   1975|460011.06|
|BAYCHESTER       ...|02  TWO FAMILY HO...|   74|    2781.46|   1954|344140.58|
|BAYCHESTER       ...|01  ONE FAMILY HO...|   77|    2854.49|   1941|242051.42|
|SCHUYLERVILLE/PEL...|03  THREE FAMILY ...|   14|    3263.86|   1928|414714.29|
|SCHUYLERVILLE/PEL...|02  TWO FAMILY HO...|   57|    3137.93|   1938|387447.02|
|SCHUYLERVILLE/PEL...|01  ONE FAMILY HO...|   36|    2751.03|   1927|344840.61|
+--------------------+--------------------+-----+-----------+-------+---------+

